In [ ]:
import numpy as np
import pandas as pd 


In [ ]:
result=pd.read_csv('/content/preprocessed_ade.csv')

In [ ]:
result

,Unnamed: 0,observation,labels
0,12606,early phase ii semi double blind study of the ...,0
1,685,cessation of mtx therapy led to complete regre...,0
2,5127,cutaneous biopsy showed hyphae and round inclu...,0
3,3158,she was treated with acyclovir and subsequentl...,1
4,10685,cisplatin was substituted and the patient achi...,0
...,...,...,...
23511,982,ticlopidine induced aplastic anemia two new c...,1
23512,14487,the risks of sumatriptan administration in pat...,0
23513,4680,however the spectrum of hosts and clinical p...,0
23514,2588,antibiotic treatment should be continued,0


In [ ]:
sentences = result['observation'].tolist() 

In [ ]:
sentences

['early phase ii semi double blind study of the new alkaline propanolamine derivative enciprazine   short communication',
 'cessation of mtx therapy led to complete regression of the tumor',
 'cutaneous biopsy showed hyphae and round inclusions stained with pas and gomori grocott within a polymorphous granuloma',
 'she was treated with acyclovir and subsequently developed vzv antigen positive zoster',
 'cisplatin was substituted and the patient achieved a complete response to therapy without further hypersensitivity',
 'severe hypomagnesemia occurred in this patient and published literature is highlighted',
 'so he underwent surgical treatment',
 'one case presented with stevens johnson syndrome   and the other had systemic papular erythema',
 'it draws the clinician is attention to unusual susceptibility to extrapyramidal side effects and suggests that if a child shows this type of susceptibility to one neuroleptic   he she may react similarly to other neuroleptics as well',
 'the inf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=1000,ngram_range=(2,2))
x=tfidf_v.fit_transform(sentences).toarray()

In [ ]:
x.shape

(23516, 1000)

In [ ]:
y=result['labels']

In [ ]:
tfidf_v.get_feature_names()[:20]

['abdominal pain',
 'about the',
 'absence of',
 'according to',
 'acute lymphoblastic',
 'acute myeloid',
 'acute myocardial',
 'acute pancreatitis',
 'acute renal',
 'added to',
 'addition of',
 'addition to',
 'administration of',
 'admitted to',
 'adverse drug',
 'adverse effect',
 'adverse effects',
 'adverse events',
 'adverse reaction',
 'adverse reactions']

In [ ]:
tfidf_v.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 1000,
 'min_df': 1,
 'ngram_range': (2, 2),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

**SPLITTING THE DATASET INTO TRAINING AND TESTING**

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2021) 

In [ ]:

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape) 

(18812, 1000)
(4704, 1000)
(18812,)
(4704,)


In [ ]:
zerocount=0
onecount=0
for i in y_train:
  if i == 0:
    zerocount+=1
  else:
    onecount+=1
print("no of negative sample in training class is:",zerocount)
print("no of positive sample in training class is:",onecount)

no of negative sample in training class is: 13335
no of positive sample in training class is: 5477


**LOGISTIC REGRESSION**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_model=LogisticRegression(max_iter=1000)

In [ ]:
log_model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
log_y_pred=log_model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score 

In [ ]:
print(confusion_matrix(y_test,log_y_pred))

[[3103  257]
 [ 729  615]]


In [ ]:
print((classification_report(y_test,log_y_pred)))

              precision    recall  f1-score   support

           0       0.81      0.92      0.86      3360
           1       0.71      0.46      0.56      1344

    accuracy                           0.79      4704
   macro avg       0.76      0.69      0.71      4704
weighted avg       0.78      0.79      0.77      4704



In [ ]:
#calculate training error 
log_y_pred_train = log_model.predict(x_train) 

In [ ]:
accuracy_score(y_train,log_y_pred_train) 

0.8102806719115458

**DECISION TREE**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_model=DecisionTreeClassifier(max_depth=5)

In [ ]:
dt_model.fit(x_train,y_train) 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dt_pred=dt_model.predict(x_test)

In [ ]:
print(classification_report(y_test,dt_pred))

              precision    recall  f1-score   support

           0       0.75      0.94      0.84      3360
           1       0.61      0.23      0.33      1344

    accuracy                           0.74      4704
   macro avg       0.68      0.58      0.58      4704
weighted avg       0.71      0.74      0.69      4704



In [ ]:
print(confusion_matrix(y_test,dt_pred))

[[3166  194]
 [1040  304]]


In [ ]:
dt_pred_train = dt_model.predict(x_train) 

In [ ]:
accuracy_score(y_train,dt_pred_train ) 

0.7351690410376356

**RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc_model=RandomForestClassifier(n_estimators=1000,max_leaf_nodes=80)

In [ ]:
rfc_model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=80, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rfc_pred=rfc_model.predict(x_test)

In [ ]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

           0       0.79      0.95      0.86      3360
           1       0.74      0.37      0.49      1344

    accuracy                           0.78      4704
   macro avg       0.77      0.66      0.68      4704
weighted avg       0.78      0.78      0.76      4704



In [ ]:
print(confusion_matrix(y_test,rfc_pred))

[[3191  169]
 [ 853  491]]


In [ ]:
rfc_train = rfc_model.predict(x_train)

In [ ]:
accuracy_score(y_train,rfc_train)

0.7955028705081862

**NAIVE BAYES**

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB

In [ ]:
gnb_model=GaussianNB()
mnb_model=MultinomialNB()

In [ ]:
gnb_model.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
mnb_model.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
gnb_pred=gnb_model.predict(x_test)

In [ ]:
mnb_pred=mnb_model.predict(x_test)

In [ ]:
print(classification_report(y_test,gnb_pred))

              precision    recall  f1-score   support

           0       0.87      0.59      0.70      3360
           1       0.43      0.77      0.55      1344

    accuracy                           0.64      4704
   macro avg       0.65      0.68      0.63      4704
weighted avg       0.74      0.64      0.66      4704



In [ ]:
print(confusion_matrix(y_test,gnb_pred))

[[1978 1382]
 [ 303 1041]]


In [ ]:
print(classification_report(y_test,mnb_pred))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86      3360
           1       0.71      0.38      0.49      1344

    accuracy                           0.78      4704
   macro avg       0.75      0.66      0.67      4704
weighted avg       0.77      0.78      0.75      4704



In [ ]:
print(confusion_matrix(y_test,mnb_pred))

[[3157  203]
 [ 840  504]]


In [ ]:
gnb_train_pred = gnb_model.predict(x_train)
mnb_train_pred = mnb_model.predict(x_train)

In [ ]:
accuracy_score(y_train,gnb_train_pred)

0.6571869019774612

In [ ]:
accuracy_score(y_train,mnb_train_pred)

0.7852966191792473

**XG BOOST** 

In [ ]:
pip install xgboost 

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model=XGBClassifier()


In [ ]:
xgb_model.fit(x_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_pred = xgb_model.predict(x_test)

In [ ]:
print(classification_report(y_test,xgb_pred))

              precision    recall  f1-score   support

           0       0.76      0.97      0.85      3360
           1       0.77      0.22      0.34      1344

    accuracy                           0.76      4704
   macro avg       0.76      0.60      0.60      4704
weighted avg       0.76      0.76      0.71      4704



In [ ]:
print(confusion_matrix(y_test,xgb_pred))

[[3273   87]
 [1049  295]]


In [ ]:
xgb_pred_train = xgb_model.predict(x_train) 

In [ ]:
accuracy_score(y_train,xgb_pred_train)  

0.7616946629810759